In [1]:
# 使用git和git-lfs克隆可变维度的文本表征模型
# git clone https://hf-mirror.com/Classical/Yinka

In [2]:
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import normalize
from sentence_transformers import SentenceTransformer

#### 加载文本表征模型

In [7]:
# 加载模型（同时获取 tokenizer）
model = SentenceTransformer("../models/Yinka")
tokenizer = model.tokenizer  # 获取底层 tokenizer

# 原始语料
corpus_texts = [
    "怎样使胡子不浓密！",
    "香港买手表哪里好",
    "在杭州手机到哪里买",
    "胡子长得太快怎么办？",
    "这个句子包含𓀀特殊字符",  # 𓀀 是古埃及象形文字，大概率是 OOV
    "正常句子123",
]

# 检查句子是否包含 OOV 字符（即 tokenizer 会拆分成 [UNK]）
def contains_oov(text):
    tokens = tokenizer.tokenize(text)
    return "[UNK]" in tokens  # 如果 tokenizer 无法识别某些字符，会返回 [UNK]

# 过滤掉包含 OOV 的句子
filtered_texts = [text for text in corpus_texts if not contains_oov(text)]
print("过滤后的句子:", filtered_texts)

# 生成嵌入向量（仅对有效句子）
vectors = model.encode(
    filtered_texts,
    normalize_embeddings=False,
    show_progress_bar=True,
    batch_size=1,
)
print("嵌入向量形状:", vectors.shape)  # (n_valid_sentences, embedding_dim)

# 降维 + 归一化（可选）
n_dims = 32
cut_vecs = normalize(vectors[:, :n_dims])

过滤后的句子: ['怎样使胡子不浓密！', '香港买手表哪里好', '在杭州手机到哪里买', '胡子长得太快怎么办？', '正常句子123']


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

嵌入向量形状: (5, 1792)


#### dbscan文本聚类

In [8]:
db = DBSCAN(eps=0.5, min_samples=2).fit(cut_vecs)
labels = db.labels_

output_df = pd.DataFrame()
output_df["文本"] = filtered_texts
output_df["聚类标签"] = labels

output_df

,文本,聚类标签
0,怎样使胡子不浓密！,0
1,香港买手表哪里好,-1
2,在杭州手机到哪里买,-1
3,胡子长得太快怎么办？,0
4,正常句子123,-1
